In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import imageio
from glob import glob
from scipy.linalg import sqrtm

# --- Constants and Configuration ---
BUFFER_SIZE = 60000
BATCH_SIZE = 256
IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_CHANNELS = 3
LATENT_DIM = 100
ITERATIONS = 20000
FID_SAMPLE_SIZE = 3000

# --- 1. Data Loading and Preparation ---
def load_emoji_dataset(data_dir):
    """
    Loads images from the specified directory and prepares the tf.data.Dataset.
    """
    if not os.path.exists(data_dir):
        print(f"Error: The directory '{data_dir}' does not exist.")
        print("Please download and unzip the EmojiOne dataset and place the '32' folder")
        print("at the correct path.")
        return None

    # Use glob to find all png files
    image_paths = glob(os.path.join(data_dir, '*.png'))
    print(f"Found {len(image_paths)} images.")

    def load_and_preprocess_image(path):
        image = tf.io.read_file(path)
        # Decode PNG, it might have 4 channels (RGBA)
        image = tf.image.decode_png(image, channels=4)
        # Drop the alpha channel
        image = image[:, :, :3]
        image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
        # Normalize the images to [-1, 1]
        image = (tf.cast(image, tf.float32) - 127.5) / 127.5
        return image

    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return dataset

# --- 2. DCGAN Model Architecture ---

def make_generator_model():
    """Builds the Generator model based on the specified architecture."""
    model = tf.keras.Sequential()
    # Input: z (noise vector)
    model.add(layers.Dense(4*4*512, use_bias=False, input_shape=(LATENT_DIM,)))
    model.add(layers.Reshape((4, 4, 512)))
    assert model.output_shape == (None, 4, 4, 512)

    # Upsampling block 1
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsampling block 2
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Output block
    model.add(layers.Conv2DTranspose(IMG_CHANNELS, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

    return model

def make_discriminator_model():
    """Builds the Discriminator model based on the specified architecture."""
    model = tf.keras.Sequential()
    # Input: 32x32x3 image
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS]))
    model.add(layers.LeakyReLU())

    # Conv block 1
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Conv block 2
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Output
    model.add(layers.Flatten())
    model.add(layers.Dense(1)) # No sigmoid for LSGAN

    return model


# --- 3. Loss Functions and Optimizers ---

def discriminator_loss(real_output, fake_output):
    """Least Squares loss for the discriminator."""
    real_loss = tf.reduce_mean(tf.math.square(real_output - 1.0))
    fake_loss = tf.reduce_mean(tf.math.square(fake_output))
    total_loss = 0.5 * (real_loss + fake_loss)
    return total_loss

def generator_loss(fake_output):
    """Least Squares loss for the generator."""
    return tf.reduce_mean(tf.math.square(fake_output - 1.0))

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# --- 4. Training Loop ---

# A single training step
@tf.function
def train_step(images, generator, discriminator):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def generate_and_save_images(model, epoch, test_input, output_dir):
    """Generates images and saves them as a grid."""
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Denormalize image from [-1, 1] to [0, 1]
        plt.imshow((predictions[i, :, :, :] + 1.0) / 2.0)
        plt.axis('off')

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    plt.savefig(os.path.join(output_dir, f'image_at_iteration_{epoch:05d}.png'))
    plt.close(fig)


def train(dataset, iterations, generator, discriminator, output_dir):
    """The main training loop."""
    seed = tf.random.normal([16, LATENT_DIM]) # Fixed noise for visualization

    data_iterator = iter(dataset.repeat())

    for iteration in range(iterations):
        start = time.time()

        # Get next batch of real images
        real_images = next(data_iterator)

        # Perform a training step
        gen_loss, disc_loss = train_step(real_images, generator, discriminator)

        # Save images every 200 iterations
        if (iteration + 1) % 200 == 0:
            generate_and_save_images(generator, iteration + 1, seed, output_dir)
            print(f'Iteration {iteration + 1}, Gen Loss: {gen_loss}, Disc Loss: {disc_loss}, Time: {time.time()-start:.2f} sec')

    # Final generation
    generate_and_save_images(generator, iterations, seed, output_dir)


# --- 5. FID Calculation ---

def calculate_fid(model, real_images, generated_images):
    """Calculates the Fréchet Inception Distance."""
    # Load InceptionV3 model pre-trained on ImageNet
    inception_model = tf.keras.applications.InceptionV3(include_top=False,
                                                        pooling='avg',
                                                        input_shape=(75,75,3)) # FID uses a minimum size of 75x75

    def preprocess_for_inception(images):
        # Resize images to 75x75 as required by InceptionV3 for FID
        images_resized = tf.image.resize(images, (75, 75), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        # InceptionV3 preprocesses from [0, 255] range
        images_rescaled = (images_resized + 1.0) * 127.5
        return tf.keras.applications.inception_v3.preprocess_input(images_rescaled)

    # Preprocess images
    real_images_preprocessed = preprocess_for_inception(real_images)
    generated_images_preprocessed = preprocess_for_inception(generated_images)

    # Calculate activations
    act1 = inception_model.predict(real_images_preprocessed)
    act2 = inception_model.predict(generated_images_preprocessed)

    # Calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # Calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # Calculate sqrt of product of cov
    covmean = sqrtm(sigma1.dot(sigma2))

    # Check and correct for imaginary numbers from sqrtm
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # Calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


# --- Main Execution ---
if __name__ == '__main__':
    # --- IMPORTANT ---
    # 1. Download the emoji dataset from one of the provided links.
    # 2. Unzip the file.
    # 3. Place the folder named '32' (from joypixels-7.0-free/png/unicode/32)
    #    into the same directory as this script, or update the path below.
    DATA_DIR = '32'
    OUTPUT_DIR = 'dcgan_training_output'

    # Load dataset
    emoji_dataset = load_emoji_dataset(DATA_DIR)

    if emoji_dataset:
        # Create models
        generator = make_generator_model()
        discriminator = make_discriminator_model()

        # Train the DCGAN
        print("\n--- Starting DCGAN Training ---")
        train(emoji_dataset, ITERATIONS, generator, discriminator, OUTPUT_DIR)
        print("\n--- Training Complete ---")

        # --- FID Evaluation ---
        print("\n--- Starting FID Evaluation ---")
        # 1. Generate images
        print(f"Generating {FID_SAMPLE_SIZE} images for FID calculation...")
        noise = tf.random.normal([FID_SAMPLE_SIZE, LATENT_DIM])
        generated_images_for_fid = generator(noise, training=False)

        # 2. Load real images
        print(f"Loading {FID_SAMPLE_SIZE} real images for FID calculation...")
        real_images_for_fid = []
        for batch in emoji_dataset.take( (FID_SAMPLE_SIZE // BATCH_SIZE) + 1 ):
            for img in batch:
                real_images_for_fid.append(img)
        real_images_for_fid = tf.stack(real_images_for_fid)[:FID_SAMPLE_SIZE]

        # 3. Calculate FID
        fid_score = calculate_fid(None, real_images_for_fid, generated_images_for_fid)
        print(f'\n---> Fréchet Inception Distance (FID): {fid_score:.2f} <---')
        print("(Lower is better)")

Found 2570 images.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Starting DCGAN Training ---
Iteration 200, Gen Loss: 0.5689087510108948, Disc Loss: 0.12533392012119293, Time: 7.46 sec
Iteration 400, Gen Loss: 0.815582275390625, Disc Loss: 0.08820968866348267, Time: 8.94 sec
Iteration 600, Gen Loss: 0.4354490339756012, Disc Loss: 0.14376750588417053, Time: 8.64 sec
Iteration 800, Gen Loss: 0.6004880666732788, Disc Loss: 0.0839281976222992, Time: 7.35 sec
Iteration 1000, Gen Loss: 0.9469025135040283, Disc Loss: 0.13228042423725128, Time: 8.73 sec


# New Section